In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os


In [2]:
# Dataset paths
train_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\train"
test_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\test"

# Parameters
img_height = 224
img_width = 224
batch_size = 16
epochs = 20  # Start with fewer epochs for observation
steps_per_epoch = 200  # Limit steps per epoch
validation_steps = 50  # Limit validation steps


In [3]:
# Enhanced data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of training data for validation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 20210 images belonging to 16 classes.
Found 5045 images belonging to 16 classes.
Found 17547 images belonging to 16 classes.


In [4]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights))


In [5]:
# Load ResNet152 with pretrained ImageNet weights
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers initially
base_model.trainable = False


In [6]:
# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                8

In [7]:
# Define callbacks for learning rate adjustment and early stopping
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]


In [8]:
# Train the model with class weights
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/20
200/200 [==============================] - 53s 227ms/step - loss: 2.9040 - accuracy: 0.0728 - val_loss: 2.7072 - val_accuracy: 0.0838 - lr: 1.0000e-04
Epoch 2/20
200/200 [==============================] - 45s 225ms/step - loss: 2.8432 - accuracy: 0.0706 - val_loss: 2.8002 - val_accuracy: 0.0350 - lr: 1.0000e-04
Epoch 3/20
200/200 [==============================] - 44s 221ms/step - loss: 2.8982 - accuracy: 0.0506 - val_loss: 2.7900 - val_accuracy: 0.0562 - lr: 1.0000e-04
Epoch 4/20
200/200 [==============================] - ETA: 0s - loss: 2.7613 - accuracy: 0.0597
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
200/200 [==============================] - 44s 220ms/step - loss: 2.7613 - accuracy: 0.0597 - val_loss: 2.7871 - val_accuracy: 0.0037 - lr: 1.0000e-04
Epoch 5/20
200/200 [==============================] - 45s 225ms/step - loss: 2.7535 - accuracy: 0.0500 - val_loss: 2.7656 - val_accuracy: 0.0288 - lr: 5.0000e-05
Epoch 6/20
200/200 [=========

In [9]:
# Unfreeze the base model layers for fine-tuning
base_model.trainable = True

# Freeze only the first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/10
200/200 [==============================] - 77s 328ms/step - loss: 2.9356 - accuracy: 0.0697 - val_loss: 9.2225 - val_accuracy: 0.0450 - lr: 1.0000e-05
Epoch 2/10
200/200 [==============================] - 63s 315ms/step - loss: 2.7150 - accuracy: 0.0847 - val_loss: 2.7722 - val_accuracy: 0.0625 - lr: 1.0000e-05
Epoch 3/10
200/200 [==============================] - 64s 321ms/step - loss: 2.6730 - accuracy: 0.0834 - val_loss: 2.7302 - val_accuracy: 0.0838 - lr: 1.0000e-05
Epoch 4/10
200/200 [==============================] - 63s 316ms/step - loss: 2.6766 - accuracy: 0.1022 - val_loss: 3.0171 - val_accuracy: 0.0275 - lr: 1.0000e-05
Epoch 5/10
 57/200 [=======>......................] - ETA: 38s - loss: 2.6365 - accuracy: 0.0888

KeyboardInterrupt: 

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")
